In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
sns_resource = boto3.resource('sns')
topic_name = 'YOUR_TOPIC_NAME'
subscribe_url = 'YOUR_SUBSCRIBE_ADDRESS'

In [ ]:
def create_topic(name):
    """
    Creates a notification topic.

    :param name: The name of the topic to create.
    :return: The newly created topic.
    """
    try:
        topic = sns_resource.create_topic(Name=name)
        logger.info("Created topic %s with ARN %s.", name, topic.arn)
    except ClientError:
        logger.exception("Couldn't create topic %s.", name)
        raise
    else:
        return topic

In [ ]:
topic = create_topic(topic_name)
print(topic)

In [ ]:
def subscribe(topic, protocol, endpoint):
    """
    Subscribes an endpoint to the topic. Some endpoint types, such as email,
    must be confirmed before their subscriptions are active. When a subscription
    is not confirmed, its Amazon Resource Number (ARN) is set to
    'PendingConfirmation'.

    :param topic: The topic to subscribe to.
    :param protocol: The protocol of the endpoint, such as 'sms' or 'email'.
    :param endpoint: The endpoint that receives messages, such as a phone number
                        (in E.164 format) for SMS messages, or an email address for
                        email messages.
    :return: The newly added subscription.
    """
    try:
        subscription = topic.subscribe(
            Protocol=protocol, Endpoint=endpoint, ReturnSubscriptionArn=True)
        logger.info("Subscribed %s %s to topic %s.", protocol, endpoint, topic.arn)
    except ClientError:
        logger.exception(
            "Couldn't subscribe %s %s to topic %s.", protocol, endpoint, topic.arn)
        raise
    else:
        return subscription

In [ ]:
subscription = subscribe(topic, 'http', subscribe_url)
print(subscription)

In [ ]:
def publish_message(topic, message, attributes):
    try:
        att_dict = {}
        for key, value in attributes.items():
            if isinstance(value, str):
                att_dict[key] = {'DataType': 'String', 'StringValue': value}
            elif isinstance(value, bytes):
                att_dict[key] = {'DataType': 'Binary', 'BinaryValue': value}
        response = topic.publish(Message=message, MessageAttributes=att_dict)
        message_id = response['MessageId']
        logger.info(
            "Published message with attributes %s to topic %s.", attributes,
            topic.arn)
    except ClientError:
        logger.exception("Couldn't publish message to topic %s.", topic.arn)
        raise
    else:
        return message_id

In [ ]:
publish_message(topic, 'Hello World!',{})

In [ ]:
def delete_topic(topic):
    """
    Deletes a topic. All subscriptions to the topic are also deleted.
    """
    try:
        topic.delete()
        logger.info("Deleted topic %s.", topic.arn)
    except ClientError:
        logger.exception("Couldn't delete topic %s.", topic.arn)
        raise
def delete_subscription(subscription):
    """
    Unsubscribes and deletes a subscription.
    """
    try:
        subscription.delete()
        logger.info("Deleted subscription %s.", subscription.arn)
    except ClientError:
        logger.exception("Couldn't delete subscription %s.", subscription.arn)
        raise

In [ ]:
delete_subscription(subscription)
delete_topic(topic)